# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
# module = os.path.abspath('/home/learner/DLA_project/src/main')
module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [3]:

manager.load_all()


Loading data from S2
	Path=C:\WESAD\S2\S2.pkl
Loading data from S3
	Path=C:\WESAD\S3\S3.pkl
Loading data from S4
	Path=C:\WESAD\S4\S4.pkl
Loading data from S5
	Path=C:\WESAD\S5\S5.pkl
Loading data from S6
	Path=C:\WESAD\S6\S6.pkl
Loading data from S7
	Path=C:\WESAD\S7\S7.pkl
Loading data from S8
	Path=C:\WESAD\S8\S8.pkl
Loading data from S9
	Path=C:\WESAD\S9\S9.pkl
Loading data from S10
	Path=C:\WESAD\S10\S10.pkl
Loading data from S11
	Path=C:\WESAD\S11\S11.pkl


In [4]:
manager.compute_features();
manager.compute_features_stress();

conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11
conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11


In [5]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)

We have 10  subjects
there are  44461  values for  a_mean
there are  44461  values for  a_std
there are  44461  values for  a_maxx
there are  44461  values for  a_maxy
there are  44461  values for  a_maxz
there are  44461  values for  e_max
there are  44461  values for  e_min
there are  44461  values for  e_mean
there are  44461  values for  e_range
there are  44461  values for  e_std
there are  44461  values for  t_max
there are  44461  values for  t_min
there are  44461  values for  t_mean
there are  44461  values for  t_range
there are  44461  values for  t_std


In [6]:
# Lets go ahead and reshape this data 
X1 = (   manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std'] )
X1 = np.reshape(X1, (len(manager.FEATURES['a_mean'])  ,15))

X2 = (   manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std'] )
X2 = np.reshape(X2, (len(manager.STRESS_FEATURES['a_mean'])  ,15))


In [7]:
# import pip

# try:
#     import keras
# except ImportError:
#     ! sudo pip3 install keras
# try:
#     import h5py
# except ImportError:
#     ! sudo pip3 install h5py
# try:
#     import ibmiotf
# except ImportError:
#     ! sudo pip3 install ibmiotf
# try:
#     import tensorflow
# except ImportError:
#     ! sudo pip3 install tensorflow

In [8]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

Using TensorFlow backend.


In [9]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, )

tf.set_random_seed(1)

session = tf.Session(graph=tf.get_default_graph(), config=session_conf)
keras.backend.set_session(session)

W0712 18:16:52.130266 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0712 18:16:52.132154 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0712 18:16:52.133272 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0712 18:16:52.209442 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables 

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [10]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [11]:
print(len(y1))
print(len(y2))

44461
23780


Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [12]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

(68241, 15)
(68241,)


In [13]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(54592, 15)
(13649, 15)
(54592,)
(13649,)


In [15]:
# Should be

# input_shape=(number of sequences=?, time_steps=None, features=15)

# target=(number of sequences, time_steps, targets)

In [16]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [17]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(54592, 1, 15)
(13649, 1, 15)
(54592,)
(13649,)


In [18]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

Need to scale the data and hope we get better results.
1. fit the scaler to the training data (fit_transform)
2. transform training data with the scaler
3. fit the model with transformed data
4. transform test data with the scaler
5. predict using model and output of (4)

In [19]:
print('Building the LSTM network...')


model = Sequential()

model.add(LSTM(32, input_shape=(1, 15), return_sequences=True))
model.add(LSTM(32, input_shape=(1, 15), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

# Note:
# Need to do return_sequences=False for the layer *before* dense
# https://stackoverflow.com/questions/51763983/error-when-checking-target-expected-dense-1-to-have-3-dimensions-but-got-array
model.add(LSTM(32, input_shape=(1, 15), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


W0712 18:16:52.469110 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Building the LSTM network...


W0712 18:16:53.113138 17776 deprecation.py:506] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.summary()

print("inputs: " , model.input_shape)

print("outputs: ", model.output_shape)

print("actual inputs: ", np.shape(X_train))

print("actual outputs: ", np.shape(y_train))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 32)             6144      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 32)             8320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 22,817
Trainable params: 22,817
Non-trainable params: 0
_________________________________________________________________
inputs:  (None, 1, 15)
outputs:  (None, 1)
actual inputs: 
actual outputs: 


In [21]:
model.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


W0712 18:16:53.936480 17776 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0712 18:16:53.980187 17776 deprecation.py:323] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
print('Training LSTM...')

batch_size = 1 # I think I want to use batch_size = 1


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

Training LSTM...
Train on 54592 samples, validate on 13649 samples
Epoch 1/5
54592/54592 [==============================] - 386s 7ms/step - loss: 0.6309 - acc: 0.6550 - val_loss: 0.6360 - val_acc: 0.6472
Epoch 2/5
54592/54592 [==============================] - 378s 7ms/step - loss: 0.6302 - acc: 0.6558 - val_loss: 0.6338 - val_acc: 0.6483
Epoch 3/5
54592/54592 [==============================] - 378s 7ms/step - loss: 0.6293 - acc: 0.6556 - val_loss: 0.6339 - val_acc: 0.6501
Epoch 4/5
54592/54592 [==============================] - 380s 7ms/step - loss: 0.6276 - acc: 0.6554 - val_loss: 0.6309 - val_acc: 0.6472
Epoch 5/5
13649/13649 [==============================] - 27s 2ms/step


In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

In [ ]:
#manager.train_model()

# manager.test_model()
